In [ ]:
from qibullet import SimulationManager
# from qibullet import RomeoVirtual
from qibullet import PepperVirtual
# from qibullet import NaoVirtual
import numpy as np
import time
import pybullet as p
import cv2
import math
from poseDetection import PoseDetection
from random import random

In [ ]:
def euler2quaternion(yaw, pitch, roll):

        qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
        qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
        qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
        qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)

        return [qx, qy, qz, qw]


In [ ]:
simulation_manager = SimulationManager()
client_id = simulation_manager.launchSimulation(gui=False)

In [ ]:
# Loading pepper
pepper = simulation_manager.spawnPepper(client_id,spawn_ground_plane=True)
pepper2 = simulation_manager.spawnPepper(client_id,translation=[1.5, 0, 0], quaternion = euler2quaternion(math.pi,0,0))
# nao = simulation_manager.spawnNao(client_id,translation=[0, 2, 0],quaternion=[0, 0, 0, 1])
# romeo = simulation_manager.spawnRomeo(client_id,translation=[0, 4, 0],spawn_ground_plane=False)

In [ ]:
p.connect(p.DIRECT)

In [ ]:
handle2 = pepper.subscribeCamera(PepperVirtual.ID_CAMERA_TOP)

In [ ]:
pd = PoseDetection(model_complexity = 2)

In [ ]:
# Controlling joints
joint_parameters = list()

for name, joint in pepper.joint_dict.items():
    if "Shoulder" in name or "Elbow" in name:
        joint_parameters.append((name,joint.getLowerLimit(),joint.getUpperLimit()))


In [ ]:
def collectData(dataset_size = 10000 , reset = False):
    
    # Accessing previous dataset
    if reset:
        print("Clearing data...")
        with open('dataset'+str(dataset_size)+'.npz', "w+") as f:
                pass
        x_data = np.empty([dataset_size,12])
        y_data = np.empty([dataset_size,8])
        step = 0
    else:
        try:
            print("Loading previous data...")
            with np.load('dataset'+str(dataset_size)+'.npz') as data:
                x_data,y_data = data['x'],data['y']
            if x_data.shape[0] != y_data.shape[0]:
                raise
            step = x_data.shape[0]
            x_data.resize((dataset_size,12))
            y_data.resize((dataset_size,8))
        except:
            print("No data found, creating new file...")
            with open('dataset'+str(dataset_size)+'.npz', "w+") as f:
                pass
            x_data = np.empty([dataset_size,12])
            y_data = np.empty([dataset_size,8])
            step = 0
    
    
    landmarks = dict()
    angles = dict()
    
    previous_time = time.time()
        try:
        while step<dataset_size:
            # Get pose from camera
            img = pepper.getCameraFrame(handle2)
            img_og = img.copy()
            results = pd.getPose(img,draw_joints =True, draw_connections = True, only_arms_connections = True)
            if results.pose_landmarks:
                for id,lm in enumerate(results.pose_landmarks.landmark):
                    if 10<id<17:
                        h, w, c = img.shape
                        x, y = int(lm.x*w) ,int(lm.y*h)
                        cv2.circle(img_og, (x,y), 3 , (0,255,0), cv2.FILLED)
                        landmarks.update({str(id)+"_x":lm.x})
                        landmarks.update({str(id)+"_y":lm.y})
            cv2.imshow("top camera", img_og)
            cv2.waitKey(1)
            
            
            # Update arm postion of actor
            current_time=time.time()
            if current_time - previous_time >5:
                for joint_parameter in joint_parameters:
                    random_position = random()*(joint_parameter[2]-joint_parameter[1])+joint_parameter[1]
                    pepper2.setAngles(joint_parameter[0],random_position,1)
                    angles.update({joint_parameter[0]:random_position})
                if (len(landmarks) == 12) and (len(angles) == 8):
                    x_data[step] = [landmarks[key] for key in landmarks]
                    y_data[step] = [angles[key] for key in angles]
                    step = step + 1
                previous_time=current_time

    except KeyboardInterrupt:
        cv2.destroyAllWindows()
    finally:
        print(x_data[:step,:])
        np.savez('dataset'+str(dataset_size)+'.npz', x=x_data[:step,:], y=y_data[:step,:])
    

In [ ]:
collectData(10000, False)

In [ ]:
# simulation_manager.stopSimulation(client_id)
